In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from qick import *
from qick_amo_v4 import *

In [2]:
# Load bitstream with custom overlay
soc = QickAmoSoc(bitfile='./qick_amo_v4.bit')
soccfg = soc

print(soccfg)

QICK running on ZCU216, software version 0.2.367

Firmware configuration (built Tue Sep 26 11:38:15 2023):

	Global clocks (MHz): tProc dispatcher timing 349.997, RF reference 245.760
	Groups of related clocks: [DAC tile 1], [DAC tile 2], [DAC tile 3]

	2 signal generator channels:
	0:	axis_sg_int4_v1 - fs=6881.280 Msps, fabric=430.080 MHz
		envelope memory: 4096 complex samples (9.524 us)
		16-bit DDS, range=1720.320 MHz
		DAC tile 1, blk 0 is 0_229 on JHC1, or QICK box DAC port 4
	1:	axis_sg_int4_v1 - fs=6881.280 Msps, fabric=430.080 MHz
		envelope memory: 4096 complex samples (9.524 us)
		16-bit DDS, range=1720.320 MHz
		DAC tile 1, blk 2 is 2_229 on JHC1, or QICK box DAC port 6

	0 readout channels:

	4 digital output pins:
	0:	PMOD0_0_LS
	1:	PMOD0_1_LS
	2:	PMOD0_2_LS
	3:	PMOD0_3_LS

	tProc: axis_tproc64x32_x8 ("v1") rev 6, program memory 1024 words, data memory 1024 words
		external start pin: None

AMO generators:
	2:	axis_signal_gen_amo_v4 - tProc output 2, NDDS = 32, parameter 

In [3]:
######################
### Sine Wave Test ###
######################
# This example will set all NDDS channels to visualize them all with a spectrum analyzer.
# Change gen_id following the numbers printed earlier to select the right signal generator.

# Refer to soccfg for generator id.
gen_id = 2
gen = soc.gens[gen_id]

#################
# Tweezers Config

# All channels off (g = 0).
gen.alloff(addr=0)
gen.alloff(addr=1)
    
# All channels on.
fmin = 20
for i in np.arange(gen.NDDS):
    gen.set_single(ch=i, addr=0, f=fmin+2*i, g=0.035)
    
fmin = 30
for i in np.arange(gen.NDDS):
    gen.set_single(ch=i, addr=1, f=fmin+2*i, g=0.035)    

###############
# tProc Program
class SineTest(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        
        # Gen AMO 0 registers.
        self.regwi(1, 10,  cfg['amo_set']                          ) # addr
        self.regwi(1, 11,  cfg['amo_wait']                         ) # wait
        self.regwi(1, 12, (cfg['amo_qsel'] << 8) + cfg['amo_phrst']) # qsel | ctrl
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.trigger(pins=[0],t=0)
        
        # Gen AMO 0.
        self.set(self.cfg['amo_gen'], 1, 10, 11, 12, 0, 0, 0)        
        
        self.synci(self.us2cycles(self.cfg['period']))

In [4]:
config={"reps"     : 1        ,
        "period"   : 500      , # us
        "amo_set"  : 1        , # address
        "amo_gen"  : gen_id   , # generator
        "amo_wait" : 0        ,
        "amo_qsel" : 1        ,
        "amo_phrst": 1        ,
       }

prog = SineTest(soccfg, config)
prog.run(soc)
#print(prog)

In [5]:
#######################
### Modulation Test ###
#######################
# This example will configure 1 interpolated signal generator to output a square-envelope pulse.
# It will also perform a simple frequency/amplitude modulation using only 1 dds for the AMO generator at addr = 0, while
# using 3 dds for the AMO generator at addr = 1. The addition of 3 sine-waves allow testing the saturation logic.
# Two parameter sets (memory locations) are used on the AMO generator and controlled from the tProc.

# Refer to soccfg for generator id.
gen_id = 2
gen = soc.gens[gen_id]

#################
# Tweezers Config

# All channels off (g = 0).
gen.alloff(addr=0)
gen.alloff(addr=1)

# Configu structure.
cfg = {}
#############################
# Sweep parameters: address 0

cfg[0] = {'channel'   : 0           , # Channel number.
          'address'   : 0           , # Memory address.
          'freq_x'    : [0,0.2,1]   , # Frequency Modulation: X axis.
          'freq_y'    : [0,3.5,0.2] , # Frequency Modulation: Y axis.
          'freq_gain' : 0.99        , # Frequency Modulation: Gain.
          'amp_x'     : [0,1]       , # Amplitude Modulation: X axis.
          'amp_y'     : [0.99,0]    , # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99        , # Amplitue Modulation: Gain.
          'phase'     : 0           , # Initial phase.
         }

#############################
# Sweep parameters: address 1
cfg[1] = {'channel'   : 0                        , # Channel number.
          'address'   : 1                        , # Memory address.
          'freq_x'    : [0,0.1,0.5,0.9,1]        , # Frequency Modulation: X axis.
          'freq_y'    : [0.1,0.11,0.15,0.19,0.2] , # Frequency Modulation: Y axis.
          'freq_gain' : 0.5                      , # Frequency Modulation: Gain.
          'amp_x'     : [0,1]                    , # Amplitude Modulation: X axis.
          'amp_y'     : [0.99, 0.5]              , # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99                     , # Amplitue Modulation: Gain.
          'phase'     : 0                        , # Initial phase.
         }

cfg[2] = {'channel'   : 1                        , # Channel number.
          'address'   : 1                        , # Memory address.
          'freq_x'    : [0,0.1,0.5,0.9,1]        , # Frequency Modulation: X axis.
          'freq_y'    : [0.1,0.11,0.15,0.19,0.2] , # Frequency Modulation: Y axis.
          'freq_gain' : 0.5                      , # Frequency Modulation: Gain.
          'amp_x'     : [0,1]                    , # Amplitude Modulation: X axis.
          'amp_y'     : [0.99, 0.5]              , # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99                     , # Amplitue Modulation: Gain.
          'phase'     : 0                        , # Initial phase.
         }

cfg[3] = {'channel'   : 2                        , # Channel number.
          'address'   : 1                        , # Memory address.
          'freq_x'    : [0,0.1,0.5,0.9,1]        , # Frequency Modulation: X axis.
          'freq_y'    : [0.1,0.11,0.15,0.19,0.2] , # Frequency Modulation: Y axis.
          'freq_gain' : 0.5                      , # Frequency Modulation: Gain.
          'amp_x'     : [0,1]                    , # Amplitude Modulation: X axis.
          'amp_y'     : [0.99, 0.5]              , # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99                     , # Amplitue Modulation: Gain.
          'phase'     : 0                        , # Initial phase.
         }

# Config.
gen.sweep_config_m(cfg)

# Write values into hardware.
gen.sweep_config_write_m(cfg)

###############
# tProc Program
class ModTest(AveragerProgram):
    def initialize(self):
        # set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"], nqz=1, mixer_freq=0)
        
        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(self.cfg["pulse_freq"])
        gain = self.cfg["pulse_gain"]
        
        length = self.us2cycles(1.0, gen_ch=self.cfg["gen_ch"])
        self.default_pulse_registers(ch=self.cfg['gen_ch'], style="const", freq=freq, gain=gain, length=length)
        
        # Gen AMO 0 registers.
        self.regwi(1, 10, 0) # addr
        self.regwi(1, 11, 1) # addr
        self.regwi(1, 12,   self.cfg['amo_wait']                                 ) # wait        
        self.regwi(1, 13, ((self.cfg['amo_qsel']+0) << 8) + ((self.cfg['amo_sat']) << 1) + self.cfg['amo_phrst']) # qsel | ctrl
        self.regwi(1, 14, ((self.cfg['amo_qsel']+1) << 8) + ((self.cfg['amo_sat']) << 1) + self.cfg['amo_phrst']) # qsel | ctrl
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.trigger(pins=[0],t=0)
        
        # Gen AMO 0.
        self.set(self.cfg['amo_gen'], 1, 10, 12, 13, 0, 0, 0)
        self.set(self.cfg['amo_gen'], 1, 11, 12, 14, 0, 0, 0)
        
        self.set_pulse_registers(ch=self.cfg["gen_ch"], phase=0, phrst=1, mode="periodic")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_start']))
        
        self.set_pulse_registers(ch=self.cfg["gen_ch"], phase=0, phrst=0, mode="oneshot")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_stop']))
        
        self.synci(2*self.us2cycles(self.cfg['pulse_stop']))

In [6]:
config={"gen_ch"     : 0, # --Fixed
        "reps"       : 15, # --Fixed
        "pulse_start": 0,
        "pulse_stop" : 150,
        "pulse_gain" : 30000, # [DAC units]
        "pulse_freq" : 0.05, # [MHz]
        
        # AMO config.
        "amo_gen"  : gen_id   , # generator
        "amo_wait" : 10       ,
        "amo_qsel" : 0        ,
        "amo_sat"  : 1        , 
        "amo_phrst": 1        ,
       }

prog = ModTest(soccfg, config)
prog.run(soc)

#print(prog)

In [7]:
##################
### Phase Test ###
##################
# This example will configure 1 interpolated signal generator to output a square-envelope pulse.
# It will also perform a simple frequency/amplitude modulation using 1 dds and 2 AMO generator outputs.
#
# AMO generators are supposed to be phase aligned. The same modulation is applied so the signals should
# be aligned all the time.
#
# Two parameter sets are used, and they are build to have the exact opposite modulation.
#
# The parameter 'amo_phrst' can be used to apply phase reset. A phase jump should be observed when the
# second modulation starts.

# Refer to soccfg for generator id.
gen_id = [2,3]

# Sweep time.
amo_sweep_t = soc.gens[gen_id[0]].cfg['SWEEP_TIME_US']

#################
# Tweezers Config

# All channels off (g = 0).
for i in gen_id:
    gen = soc.gens[i]
    
    gen.alloff(addr=0)
    gen.alloff(addr=1)

    # Configu structure.
    cfg = {}

    #############################
    # Sweep parameters: address 0

    cfg[0] = {'channel'   : 0           , # Channel number.
              'address'   : 0           , # Memory address.
              'freq_x'    : [0,0.2,1]   , # Frequency Modulation: X axis.
              'freq_y'    : [0,2.5,3.2] , # Frequency Modulation: Y axis.
              'freq_gain' : 0.99        , # Frequency Modulation: Gain.
              'amp_x'     : [0,1]       , # Amplitude Modulation: X axis.
              'amp_y'     : [0.99, 0.5] , # Amplitude Modulation: Y axis.
              'amp_gain'  : 0.99        , # Amplitue Modulation: Gain.              
              'phase'     : 0           , # Initial phase.
             }

    #############################
    # Sweep parameters: address 1
    cfg[1] = {'channel'   : 0           , # Channel number.
              'address'   : 1           , # Memory address.
              'freq_x'    : [0,0.8,1]   , # Frequency Modulation: X axis.
              'freq_y'    : [3.2,2.5,0] , # Frequency Modulation: Y axis.
              'freq_gain' : 0.99        , # Frequency Modulation: Gain.
              'amp_x'     : [0,1]       , # Amplitude Modulation: X axis.
              'amp_y'     : [0.5, 0.99] , # Amplitude Modulation: Y axis.
              'amp_gain'  : 0.99        , # Amplitue Modulation: Gain.                            
              'phase'     : 0           , # Initial phase.
             }

    # Config.
    gen.sweep_config_m(cfg)

    # Write values into hardware.
    gen.sweep_config_write_m(cfg)

###############
# tProc Program
class PhaseTest(AveragerProgram):
    def initialize(self):
        # set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"], nqz=1, mixer_freq=0)
        
        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(self.cfg["pulse_freq"])
        gain = self.cfg["pulse_gain"]
        
        length = self.us2cycles(1.0, gen_ch=self.cfg["gen_ch"])
        self.default_pulse_registers(ch=self.cfg['gen_ch'], style="const", freq=freq, gain=gain, length=length)
        
        # Gen AMO 0 registers.
        self.regwi(1, 10, 0                                 ) # addr
        self.regwi(1, 11, 1                                 ) # addr
        self.regwi(1, 12, 1                                 ) # wait        
        self.regwi(1, 13, ((1 << 8) + self.cfg["amo_phrst"])) # qsel | ctrl
        
        # Delay between AMO pulses.
        self.regwi(1, 14, self.us2cycles(self.cfg["amo_delay"]))
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.trigger(pins=[0],t=0)
        
        # Gen AMO 0.
        for i in self.cfg['amo_gens']:
            self.set(i, 1, 10, 12, 13, 0, 0, 0)
            self.set(i, 1, 11, 12, 13, 0, 0, 14)
        
        self.set_pulse_registers(ch=self.cfg["gen_ch"], phase=0, phrst=1, mode="periodic")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_start']))
        
        self.set_pulse_registers(ch=self.cfg["gen_ch"], phase=0, phrst=0, mode="oneshot")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_stop']))
        
        self.synci(2*self.us2cycles(self.cfg['pulse_stop']))

In [8]:
config={"gen_ch"     : 0      , # --Fixed
        "reps"       : 1      , # --Fixed
        "pulse_start": 0      ,
        "pulse_stop" : 150    ,
        "pulse_gain" : 30000  , # [DAC units]
        "pulse_freq" : 0.05   , # [MHz]
        
        # AMO config.
        "amo_gens"   : gen_id               , # generator
        "amo_delay"  : amo_sweep_t + 10.2   , # delay between AMO pulses.
        "amo_phrst"  : 1                    , # phase reset.
       }

prog = PhaseTest(soccfg, config)
prog.run(soc)

#print(prog)

In [9]:
####################
### Fourier Test ###
####################
# This example is useful for testing phase alignment of the AMO generator block.
#
# The Fourier Coefficients of a Square and Sawtooth wave are set on two different sets.
#
# The tProc program will run either set, and the generator should synthesize that wave.
#

# Refer to soccfg for generator id.
gen_id = 4
gen = soc.gens[gen_id]

#################
# Tweezers Config

# All channels off (g = 0).
gen.alloff(addr=0) # square wave.
gen.alloff(addr=1) # sawtooth
    
# Number of Fourier coefficients.
Nf = 10

# Frequency/amplitude.
fsq = 10
asq = 0.99

# Square wave coefficients.
cfg = {}
for i in np.arange(Nf):
    nn = 2*i+1
    aa = asq/nn
    ff = fsq*nn
    cfg[i] = {'channel'   : i          , # Channel number.
              'address'   : 0          , # Memory address.
              'freq_x'    : [0,1]      , # Frequency Modulation: X axis.
              'freq_y'    : [ff,ff]    , # Frequency Modulation: Y axis.
              'amp_x'     : [0,1]      ,
              'amp_y'     : [aa, 0*aa] , # Amplitude Modulation Points.
              'phase'     : 0          , # Initial phase.
             }
    
    # Config.
    gen.sweep_config_m(cfg)
    
    # Write values into hardware.
    gen.sweep_config_write_m(cfg)
    
# Sawtooth wave coefficients.
cfg = {}
for i in np.arange(Nf):
    nn = i+1
    aa = -asq/(np.pi*nn)
    ff = fsq*nn
    cfg[i] = {'channel'   : i          , # Channel number.
              'address'   : 1          , # Memory address.
              'freq_x'    : [0,1]      , # Frequency Modulation: X axis.
              'freq_y'    : [ff,ff]    , # Frequency Modulation: Y axis.
              'amp_x'     : [0,1]      , # Amplitude Modulation: X axis.
              'amp_y'     : [aa, 0*aa] , # Amplitude Modulation: Y axis.
              'phase'     : 0          , # Initial phase.
             }
    
    # Config.
    gen.sweep_config_m(cfg)
    
    # Write values into hardware.
    gen.sweep_config_write_m(cfg)    

###############
# tProc Program
class FourierTest(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        
        # Gen AMO 0 registers.
        self.regwi(1, 10, cfg['amo_set']) # address
        self.regwi(1, 11, (cfg['amo_qsel'] << 8) + 1) # qsel | ctrl
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.trigger(pins=[0],t=0)
        
        # Gen AMO 0.
        self.set(self.cfg['amo_gen'], 1, 10, 0, 11, 0, 0, 0)        
        
        self.synci(self.us2cycles(self.cfg['period']))

In [10]:
config={"reps"     : 1        ,
        "period"   : 500      , # us
        "amo_set"  : 0        , # address
        "amo_gen"  : gen_id   , # generator
        "amo_qsel" : 0        ,
       }

prog = FourierTest(soccfg, config)
prog.run(soc)
#print(prog)

In [11]:
#################################################
### Higher Order Amplitude/Auxiliary Out Test ###
#################################################

# Refer to soccfg for generator id.
gen_id = 4
gen = soc.gens[gen_id]

#################
# Tweezers Config

# All channels off (g = 0).
gen.alloff(addr=0)
gen.alloff(addr=1)

# Configu structure.
cfg = {}
#############################
# Sweep parameters: address 0

cfg[0] = {'channel'   : 0               , # Channel number.
          'address'   : 0               , # Memory address.
          'freq_x'    : [0,0.2,1]       , # Frequency Modulation: X axis.
          'freq_y'    : [0,3.5,0.2]     , # Frequency Modulation: Y axis.
          'freq_gain' : 0.99            , # Frequency Modulation: Gain.
          'amp_x'     : [0,0.2, 0.8,1]  , # Amplitude Modulation: X axis.
          'amp_y'     : [0,0.1,0.9,0.99], # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99            , # Amplitue Modulation: Gain.
          'phase'     : 0               , # Initial phase.
         }

cfg[1] = {'channel'   : 31               , # Channel number.
          'address'   : 0               , # Memory address.
          'freq_x'    : [0,0.2,1]       , # Frequency Modulation: X axis.
          'freq_y'    : [0,3.5,0.2]     , # Frequency Modulation: Y axis.
          'freq_gain' : 0.99            , # Frequency Modulation: Gain.
          'amp_x'     : [0,0.2, 0.8,1]  , # Amplitude Modulation: X axis.
          'amp_y'     : [0.01,0.1,0.9,0.99], # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99            , # Amplitue Modulation: Gain.
          'phase'     : 0               , # Initial phase.
         }

#############################
# Sweep parameters: address 1
cfg[2] = {'channel'   : 0                        , # Channel number.
          'address'   : 1                        , # Memory address.
          'freq_x'    : [0,0.1,0.5,0.9,1]        , # Frequency Modulation: X axis.
          'freq_y'    : [0.1,0.11,0.15,0.19,0.2] , # Frequency Modulation: Y axis.
          'freq_gain' : 0.5                      , # Frequency Modulation: Gain.
          'amp_x'     : [0,1]                    , # Amplitude Modulation: X axis.
          'amp_y'     : [0.99, 0.5]              , # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99                     , # Amplitue Modulation: Gain.
          'phase'     : 0                        , # Initial phase.
         }

cfg[3] = {'channel'   : 1                        , # Channel number.
          'address'   : 1                        , # Memory address.
          'freq_x'    : [0,0.1,0.5,0.9,1]        , # Frequency Modulation: X axis.
          'freq_y'    : [0.1,0.11,0.15,0.19,0.2] , # Frequency Modulation: Y axis.
          'freq_gain' : 0.5                      , # Frequency Modulation: Gain.
          'amp_x'     : [0,1]                    , # Amplitude Modulation: X axis.
          'amp_y'     : [0.99, 0.5]              , # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99                     , # Amplitue Modulation: Gain.
          'phase'     : 0                        , # Initial phase.
         }

cfg[4] = {'channel'   : 2                        , # Channel number.
          'address'   : 1                        , # Memory address.
          'freq_x'    : [0,0.1,0.5,0.9,1]        , # Frequency Modulation: X axis.
          'freq_y'    : [0.1,0.11,0.15,0.19,0.2] , # Frequency Modulation: Y axis.
          'freq_gain' : 0.5                      , # Frequency Modulation: Gain.
          'amp_x'     : [0,1]                    , # Amplitude Modulation: X axis.
          'amp_y'     : [0.99, 0.5]              , # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99                     , # Amplitue Modulation: Gain.
          'phase'     : 0                        , # Initial phase.
         }

cfg[5] = {'channel'   : 31                        , # Channel number.
          'address'   : 1                        , # Memory address.          
          'freq_y'    : [0,1]                    , # NOT USED.          
          'amp_x'     : [0,0.3,0.7,1]            , # Amplitude Modulation: X axis.
          'amp_y'     : [0.99,0.7,0.3,0.2]       , # Amplitude Modulation: Y axis.
          'amp_gain'  : 0.99                     , # Amplitue Modulation: Gain.
         }

# Config.
gen.sweep_config_m(cfg)

# Write values into hardware.
gen.sweep_config_write_m(cfg)

###############
# tProc Program
class ModTest(AveragerProgram):
    def initialize(self):
        # set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"], nqz=1, mixer_freq=0)
        
        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(self.cfg["pulse_freq"])
        gain = self.cfg["pulse_gain"]
        
        length = self.us2cycles(1.0, gen_ch=self.cfg["gen_ch"])
        self.default_pulse_registers(ch=self.cfg['gen_ch'], style="const", freq=freq, gain=gain, length=length)
        
        # Gen AMO 0 registers.
        self.regwi(1, 10, 0) # addr
        self.regwi(1, 11, 1) # addr
        self.regwi(1, 12,   self.cfg['amo_wait']                                 ) # wait        
        self.regwi(1, 13, ((self.cfg['amo_qsel']+0) << 8) + ((self.cfg['amo_sat']) << 1) + self.cfg['amo_phrst']) # qsel | ctrl
        self.regwi(1, 14, ((self.cfg['amo_qsel']+1) << 8) + ((self.cfg['amo_sat']) << 1) + self.cfg['amo_phrst']) # qsel | ctrl
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.trigger(pins=[0],t=0)
        
        # Gen AMO 0.
        self.set(self.cfg['amo_gen'], 1, 10, 12, 13, 0, 0, 0)
        self.set(self.cfg['amo_gen'], 1, 11, 12, 14, 0, 0, 0)
        
        self.set_pulse_registers(ch=self.cfg["gen_ch"], phase=0, phrst=1, mode="periodic")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_start']))
        
        self.set_pulse_registers(ch=self.cfg["gen_ch"], phase=0, phrst=0, mode="oneshot")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_stop']))
        
        self.synci(2*self.us2cycles(self.cfg['pulse_stop']))

In [12]:
config={"gen_ch"     : 0, # --Fixed
        "reps"       : 15, # --Fixed
        "pulse_start": 0,
        "pulse_stop" : 150,
        "pulse_gain" : 30000, # [DAC units]
        "pulse_freq" : 0.05, # [MHz]
        
        # AMO config.
        "amo_gen"  : gen_id   , # generator
        "amo_wait" : 10       ,
        "amo_qsel" : 0        ,
        "amo_sat"  : 1        , 
        "amo_phrst": 1        ,
       }

prog = ModTest(soccfg, config)
prog.run(soc)

#print(prog)

In [13]:
##################################
### Zero-Order Modulation Test ###
##################################

# Refer to soccfg for generator id.
gen_id = 4
gen = soc.gens[gen_id]

#################
# Tweezers Config

# All channels off (g = 0).
gen.alloff(addr=0)

# Configu structure.
cfg = {}
#############################
# Sweep parameters: address 0

cfg[0] = {'channel'   : 0    , # Channel number.
          'address'   : 0    , # Memory address.
          'freq_x'    : [0]  , # Frequency Modulation: X axis.
          'freq_y'    : [10] , # Frequency Modulation: Y axis.
          'amp_x'     : [0]  , # Amplitude Modulation: X axis.
          'amp_y'     : [0.2], # Amplitude Modulation: Y axis.
          'phase'     : 0    , # Initial phase.
         }

# Config.
gen.sweep_config_m(cfg)

# Write values into hardware.
gen.sweep_config_write_m(cfg)

###############
# tProc Program
class ModTest(AveragerProgram):
    def initialize(self):
        # set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"], nqz=1, mixer_freq=0)
        
        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(self.cfg["pulse_freq"])
        gain = self.cfg["pulse_gain"]
        
        length = self.us2cycles(1.0, gen_ch=self.cfg["gen_ch"])
        self.default_pulse_registers(ch=self.cfg['gen_ch'], style="const", freq=freq, gain=gain, length=length)
        
        # Gen AMO 0 registers.
#         ctrl_ = (self.cfg['amo_qsel'] << 8) + (self)
        self.regwi(1, 10, 0) # addr
        self.regwi(1, 11,   self.cfg['amo_wait']                                 ) # wait        
        self.regwi(1, 12, ((self.cfg['amo_qsel']+0) << 8) + ((self.cfg['amo_sat']) << 1) + self.cfg['amo_phrst']) # qsel | ctrl
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.trigger(pins=[0],t=0)
        
        # Gen AMO 0.
        self.set(self.cfg['amo_gen'], 1, 10, 11, 12, 0, 0, 0)
        
        self.set_pulse_registers(ch=self.cfg["gen_ch"], phase=0, phrst=1, mode="periodic")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_start']))
        
        self.set_pulse_registers(ch=self.cfg["gen_ch"], phase=0, phrst=0, mode="oneshot")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_stop']))
        
        self.synci(2*self.us2cycles(self.cfg['pulse_stop']))

In [14]:
config={"gen_ch"     : 0, # --Fixed
        "reps"       : 15, # --Fixed
        "pulse_start": 0,
        "pulse_stop" : 150,
        "pulse_gain" : 30000, # [DAC units]
        "pulse_freq" : 0.05, # [MHz]
        
        # AMO config.
        "amo_gen"  : gen_id   , # generator
        "amo_wait" : 10       ,
        "amo_qsel" : 0        ,
        "amo_sat"  : 1        , 
        "amo_phrst": 0        ,
       }

prog = ModTest(soccfg, config)
prog.run(soc)

#print(prog)